In [ ]:
refgrid = '/home/jasp/autocls_exp3/pxs_rst/refpxgroups_06x06_4.tif'

lulcmaps = {
    'clcp2018' : '/mnt/d/autocls_exp3/lulc_06x06/clcpp_bb2018_06x06.tif',
    'cos2018'  : '/mnt/d/autocls_exp3/lulc_06x06/cos_2018_06x06.tif',
    'cosc2021' : '/mnt/d/autocls_exp3/lulc_06x06/cosc_2021_06x06.tif',
    'ua2018'   : '/mnt/d/autocls_exp3/lulc_06x06/ua_2018_06x06.tif'
}

lulc_col = {
    'clcp2018' : 'Value',
    'cos2018'  : 'COS18n4_C',
    'cosc2021' : 'Value',
    'ua2018'   : 'code_2018'
}

lulc_mixed = {
    'clcp2018' : '/home/jasp/autocls_exp3/px_mixed/mixed3_clcp2018.tif',
    'cos2018'  : '/home/jasp/autocls_exp3/px_mixed/mixed3_cos2018.tif',
    'cosc2021' : '/home/jasp/autocls_exp3/px_mixed/mixed3_cosc2021.tif',
    'ua2018'   : '/home/jasp/autocls_exp3/px_mixed/mixed3_ua2018.tif'
}

pure_px_tbl = '/home/jasp/autocls_exp3/px_classes'
pure_bname = 'pxclass4'

mixed_px_rst = '/home/jasp/autocls_exp3/px_mixed'
mixed_bname = 'mixed4'

db = 'gthruth_valeflores_v3'

In [ ]:
import os
import numpy as np
import datetime as dt

from glass.pys.tm import now_as_str
from glass.wenv.grs import run_grass
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.zon.grs import rstatszonal
from glass.rst.local.grs import combine_report_to_df, grs_combine
from glass.rst.alg import grsrstcalc
from glass.rd import tbl_to_obj
from glass.wt import obj_to_tbl
from glass.sql.q import q_to_obj
from glass.dtt.mge.pd import merge_df
from glass.prop.prj import get_epsg

In [ ]:
time_a = dt.datetime.now().replace(microsecond=0)

In [ ]:
epsg = get_epsg(refgrid)

In [ ]:
# Get Lulc Raster Tables

lulc_rst_tbls = {}

for m in lulc_col:
    lulc_rst_tbls[m] = q_to_obj(db, (
        "SELECT mtbl.classid, mtbl.code, mtbl.classname, jtbl.slug "
        "FROM mapclasses AS mtbl "
        "LEFT JOIN refmaps AS jtbl "
        "ON mtbl.mapfk = jtbl.mapid "
        f"WHERE jtbl.slug = '{m}'"
    ))
    
    if lulc_col[m] != 'Value':
        _df = tbl_to_obj(f'{lulcmaps[m]}.vat.dbf')

        lulc_rst_tbls[m] = lulc_rst_tbls[m].merge(
            _df, how='right', left_on='code',
            right_on=lulc_col[m]
        )

        lulc_rst_tbls[m].drop(['Count', lulc_col[m]], axis=1,  inplace=True)
        lulc_rst_tbls[m].rename(columns={'Value' : 'rvalue'}, inplace=True)

    else:
        lulc_rst_tbls[m]['rvalue'] = lulc_rst_tbls[m]['code'].astype(int)

In [ ]:
lulc_rst_tbls

In [ ]:
# Create GRASS GIS Session
loc = now_as_str(utc=True)
ws = pure_px_tbl

gb = run_grass(ws, location=loc, srs=refgrid)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
# Import Raster's

grsref = rst_to_grs(refgrid, as_cmd=True)

grslulc = [rst_to_grs(
    lulcmaps[r], grst=r, as_cmd=True
) for r in lulcmaps]

for m in lulc_mixed:
    if not lulc_mixed[m]:
        continue

    lulc_mixed[m] = rst_to_grs(lulc_mixed[m], as_cmd=True)

In [ ]:
pure_cells_dfs = []
cells_more_one_cls = []

for _r in grslulc:
    if _r in lulc_mixed and lulc_mixed[_r]:
        # Ignore pure cells
        r = grsrstcalc(
            f"{_r} * {lulc_mixed[_r]}",
            f"{_r}_mixed", ascmd=True
        )

    else:
        r = _r
    
    cls_count = rstatszonal(grsref, r, 'range', f'count_{r}')

    # Select cells with one class only
    one_cls = grsrstcalc(
        f"if({cls_count} > 0, null(), {r})",
        f"onecls_{_r}", ascmd=True
    )

    # Combine
    cmb_onecls = grs_combine([grsref, one_cls], f'cmb_{_r}')

    cmb_table = combine_report_to_df(
        cmb_onecls, ['pxid', 'lulc_class_value'],
        otxt=os.path.join(ws, loc, f'{cmb_onecls}.txt')
    )

    cmb_table = cmb_table.merge(
        lulc_rst_tbls[_r], how='left',
        left_on='lulc_class_value',
        right_on='rvalue'
    )

    _cmb_tbl = cmb_table[['pxid', 'classid']]

    _cmb_tbl.rename(columns={'classid' : 'clsid'}, inplace=True)
    
    #pure_cells_dfs.append(_cmb_tbl)

    # Select cells with more than one class
    more_cls = grsrstcalc(
        f"if({cls_count} > 0, 1, null())",
        f"mtpcls_{_r}", ascmd=True
    )
    #cells_more_one_cls.append(more_cls)

    # Export table
    obj_to_tbl(_cmb_tbl, os.path.join(pure_px_tbl, f"{pure_bname}_{_r}.xlsx"))

    # Export raster
    grs_to_rst(
        more_cls, os.path.join(mixed_px_rst, f'{mixed_bname}_{_r}.tif'),
        as_cmd=True, rtype=int
    )

In [ ]:
time_b = dt.datetime.now().replace(microsecond=0)

In [ ]:
print(time_b - time_a)